# Exploring The Book Thief(Movie) Sentiment Analysis using Bilinear Logistic Regression
In this notebook the following shall be done:
* Obtain labelled training data from Kaggle
* Define a cleaning function for text cleaning
* Choose a model(Bilinear Logistic Regression), split the data into test and train data
* Build and train the model
* Scrape the BookThief Reviews from IMDB
* Predict the polarity of the reviews

## Sentiment Analysis
Sentiment analysis is an NLP process of identifying the tone expressed in a text. Is it positive,negative or neutral. 
Opinion mining is especially useful for feedback on how customers feel about a brand or product and helps a business understand better customer needs.

## NLP: Natural Language Processing
This is a branch of machine learning that allows a computer to read, understand and draw meaning from human understandable text. We understand words and computers understand text. Human readable text has to undergo a couple of transformations before machines can make sense of it. These include:
* Segmentation - This is the breaking down of a document into its constituent sentences
* Tokenization - Breaking down of sentences into words
* Removal of Stopwords - Stopwords are a list of words to be filtered out from the document as they are insignificant due to their popularity when it comes to natural language processing.They include prepositions, pronouns, conjunctions etc.
*Lemmatization - This is the process of grouping together inflected forms of a word so they can be analysed as a single word. Example: Is, Are and Am all come under the verb 'to be' but in different persons.

## Binary Logistic Regression to get the tone
There are a variety of models that can be used to get the tone in a text. Some include SVMs, Random Forests and Naive Bayes. For this project, we will be identifying one of two tones, negative or positive.

Logistic regression predicts the probability of an event or outcome. The possible outcomes being categorical -meaning they are stored and identified based on names or labels given to them. In our case, we have prelabelled data where positive = 1 and negative = 0.

In [1]:
#importing the pandas library, reading and exploring the dataset
import pandas as pd
dataset = pd.read_csv("labeledTrainData.tsv", sep='\t', index_col=None)
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         25000 non-null  object
 1   sentiment  25000 non-null  int64 
 2   review     25000 non-null  object
dtypes: int64(1), object(2)
memory usage: 586.1+ KB


In [2]:
#exploring the dataset 
dataset.head()



,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [3]:
#exploring the number of sentiments by count
dataset['sentiment'].nunique()


2

In [4]:
#exploring the distribution number of sentiments by count
dataset['sentiment'].value_counts()

1    12500
0    12500
Name: sentiment, dtype: int64

In [5]:
# and dropping the unique_id column
dataset = dataset.dropna()
dataset = dataset.drop(dataset.columns[0], axis=1)
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   sentiment  25000 non-null  int64 
 1   review     25000 non-null  object
dtypes: int64(1), object(1)
memory usage: 390.8+ KB


In [6]:
#importing libraries that will aid in text cleaning and declaring regexes of items for removal
import re
import string

html_tags = re.compile(r'<[^<]+?>')
emojis = re.compile(
    "(["
    "\U0001F1E0-\U0001F1FF"  # flags (iOS)
    "\U0001F300-\U0001F5FF"  # symbols & pictographs
    "\U0001F600-\U0001F64F"  # emoticons
    "\U0001F680-\U0001F6FF"  # transport & map symbols
    "\U0001F700-\U0001F77F"  # alchemical symbols
    "\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
    "\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
    "\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
    "\U0001FA00-\U0001FA6F"  # Chess Symbols
    "\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
    "\U00002702-\U000027B0"  # Dingbats
    "])"
)

In [7]:
# import libraries that will allow for stopword removal, tokenization and lemmatization
from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer().lemmatize
wordpunct_tokenize = WordPunctTokenizer().tokenize
en_stop = set(stopwords.words('english'))
print(en_stop)


{'them', "needn't", 'me', 'into', "it's", 'very', "mustn't", 'over', 'nor', 'yourself', 'didn', 'll', 'does', 'has', 'where', 'more', 'd', 'through', 'its', 'hadn', 'because', 'shan', 'hasn', 'in', "you'd", 'out', 'now', 'is', 'or', 'are', "haven't", 'these', 'this', "doesn't", 'from', 'haven', 'will', 'the', 'just', 'what', 'was', 'do', 'once', 'him', 'her', 'further', 'that', "you've", 'doing', 'so', 'such', 'but', "aren't", 'for', 'too', 'of', 'why', 'any', 'ourselves', 've', 'isn', 'whom', 'my', "mightn't", 'herself', "shouldn't", 're', 'it', 'no', "isn't", "wouldn't", "hadn't", "you'll", 'at', 'against', "you're", 'while', 'a', 'not', 'about', "should've", 'after', 'he', 'how', 'o', "she's", 'am', 'under', 'by', 'ours', 'here', 'than', 'if', "didn't", 'aren', 'she', 'shouldn', 'you', 'above', 'needn', 'own', 'who', 'all', "don't", 'himself', 'theirs', 'itself', "that'll", 'yours', 'there', 'each', 'had', 'should', 'having', 'won', 'which', 'mightn', 'then', 'mustn', 'when', 'to', 

In [8]:
#creating a function that cleans the data takes a dataframe as an argument
def cleaning_tool(dataframe):
    '''
    This function cleans the dataframe
    '''
    #removes html tags
    dataframe['review'] = dataframe['review'].apply(lambda words: re.sub(html_tags, ' ', words)) 
    #removes emojis  
    dataframe['review'] = dataframe['review'].apply(lambda words: re.sub(emojis, ' ', words))  
    #removes digits [0-9]    
    dataframe['review'] = dataframe['review'].apply(lambda words: ''.join([x for x in words if not x.isdigit()])) 
    #removes   non ASCII characters
    dataframe['review'] = dataframe['review'].apply(lambda words: ''.join([x for x in words if x.isascii()]))
    #converts text into lowercase  
    dataframe['review'] = dataframe['review'].apply(lambda words: [str(words).lower()])
    #does word tokenization  
    dataframe['review'] = dataframe['review'].apply( lambda words: wordpunct_tokenize(str(words)))
    #removes punctuation marks 
    dataframe['review'] = dataframe['review'].apply(lambda words: ''.join([str(words).translate( str.maketrans('', '', string.punctuation))]))
    #word lemmatization   
    dataframe['review'] = dataframe['review'].apply(lambda words: lemmatizer(str(words)))
    # removes stopwords   
    dataframe['review'] = dataframe['review'].apply( lambda words: ' '.join([x for x in str(words).split() if x not in (en_stop)]))
       

    return dataframe

# applying the cleaning tool to the labelled data dataframe
data = cleaning_tool(dataset)
data.head()

,sentiment,review
0,1,stuff going moment mj started listening music ...
1,1,classic war worlds timothy hines entertaining ...
2,0,film starts manager nicholas bell giving welco...
3,0,must assumed praised film greatest filmed oper...
4,1,superbly trashy wondrously unpretentious explo...


In [9]:
#Defining the dependent and independent variables
Y = data['sentiment']
X = data['review']

In [10]:
#Text vectorization
from sklearn.feature_extraction.text import TfidfVectorizer
reviews_vec = TfidfVectorizer()
X_data = reviews_vec.fit_transform(X)

In [11]:
# creating a dataframe for use in feature comparison
X_data_array = X_data.toarray()
vocab = reviews_vec.get_feature_names_out()
X_data_df = pd.DataFrame(X_data_array, columns=vocab)
X_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Columns: 73617 entries, aa to zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz
dtypes: float64(73617)
memory usage: 13.7 GB


In [12]:
# splitting the dataset into train and test data
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    X_data, Y, test_size=0.20, random_state=12)


In [13]:
# building the model
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(x_train, y_train)


LogisticRegression()

In [14]:
#plot the sigmoid curve

In [15]:
# obtaining the model accuracy
lr.score(x_test, y_test)


0.887

## Section 2 : Scraping the Movie Reviews from Rotten Tomatoes and Feeding it into our Model for Sentiment Analysis


In [16]:
#importing beautifulsoup and requests library for webscraping
from bs4 import BeautifulSoup
import requests

In [17]:
# declaring necessary urls and setting up to web-scrape
start_url = 'https://www.imdb.com/title/tt0816442/reviews?ref_=tt_urv'


In [18]:
# scraping reviews from the first page
page = requests.get(start_url)
soup = BeautifulSoup(page.text, "html.parser")
list_of_reviews = [reviews.get_text() for reviews in soup.find_all(
            'div', class_="text show-more__control")]

# appending data to dataframe
columns = ['review']
book_thief = pd.concat([pd.DataFrame([i], columns=columns)
                  for i in list_of_reviews], ignore_index=True)
book_thief.head()

,review
0,Those familiar with the 2005 award winning and...
1,No extended fight scenes. No unnecessary pyrot...
2,"I do not know the book. But the film, for its ..."
3,Many books have been deemed 'unfilmable' - but...
4,"""The Book Thief"" is certainly a rare kind of f..."


In [19]:
# cleaning the dataset
book_thief = cleaning_tool(book_thief)
book_thief.head()


,review
0,familiar award winning best selling novel aust...
1,extended fight scenes unnecessary pyrotechnics...
2,know book film beautiful simplicity useful hig...
3,many books deemed unfilmable anyone read marku...
4,book thief certainly rare kind film day gleams...


In [20]:
# vectorizing the reviews
bookthief_revs = book_thief['review']
reviews_vec = TfidfVectorizer()
bookthief_data = reviews_vec.fit_transform(bookthief_revs)

In [21]:
# creating a dataframe for feature comparison
bookthief_data_array = bookthief_data.toarray()
bookthiefvocab = reviews_vec.get_feature_names_out()
bookthief_data_df = pd.DataFrame(bookthief_data_array, columns=bookthiefvocab)
bookthief_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Columns: 1926 entries, abandoned to zusak
dtypes: float64(1926)
memory usage: 376.3 KB


In [22]:
# accounting for missing vocabulary
not_exist_vocab = [v for v in X_data_df.columns.tolist()
                   if v not in bookthief_data_df]
bookthief_data_df = bookthief_data_df.reindex(
    columns=bookthief_data_df.columns.tolist() + not_exist_vocab)
bookthief_data_df = bookthief_data_df.fillna(0)
bookthief_data_df = bookthief_data_df[X_data_df.columns.tolist()]
bookthief_data_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Columns: 73617 entries, aa to zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz
dtypes: float64(73617)
memory usage: 14.0 MB


In [23]:
# converting back to sparse matrix
from scipy import sparse
bookthief_data_array2 = bookthief_data_df.to_numpy()
bookthief_data = sparse.csr_matrix(bookthief_data_array2)

In [24]:
# predicting the sentiment score
predictedvalues = lr.predict(bookthief_data)

In [25]:
# Listing the number of reviews as classified by polarity
print(f'Number of Positive Reviews : {list(predictedvalues).count(1)}')
print(f'Number of Negative Reviews : {list(predictedvalues).count(0)}')


Number of Positive Reviews : 21
Number of Negative Reviews : 4


In [28]:
#Comparing with the actual ranking
page = requests.get(start_url)
soup = BeautifulSoup(page.text, "html.parser")
list_of_ratings = [ratings.get_text() for ratings in soup.select('div.lister-item:nth-child(n) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > span:nth-child(1) > span:nth-child(2)')]
           

# appending data to dataframe
columns = ['ratings']
book_thief_ratings = pd.concat([pd.DataFrame([i], columns=columns)
                  for i in list_of_ratings], ignore_index=True)
book_thief_ratings.head()

,ratings
0,9
1,10
2,7
3,8
4,10


In [32]:
#determining the value counts of the score ratings
book_thief_ratings.ratings.value_counts()

8     8
9     5
10    3
7     3
3     1
6     1
2     1
5     1
Name: ratings, dtype: int64

## Conclusion
As seen from the page, the ratings are done on a scale of 1 to 10, with values less than or equal to 5 being ranked negative, while those greater than 5 as positive
From the page, 21 had a score of 6 and above, while 3 had a score of 5 and below.
It is worth noting that two reviews did not give a rating score.
In comparison, the model has done pretty well
